In [1]:
import pandas as pd

In [6]:
df = pd.read_excel("FuncionariosVigentesOctubre2024-DSSM.xlsx",engine='openpyxl')

In [40]:
import pandas as pd
import psycopg2
from psycopg2 import sql

# Cargar el archivo de Excel
file_path = 'FuncionariosVigentesOctubre2024-DSSM.xlsx'
df = pd.read_excel(file_path, sheet_name='VigentesOctubre2024DSSM')

# Conectar a la base de datos de PostgreSQL
connection = psycopg2.connect(
    dbname="PyCG",
    user="postgres",
    password="fede0628",
    host="localhost",
    port="5432"
)
cursor = connection.cursor()

# Insertar datos en la tabla persona a partir del DataFrame
for _, row in df.iterrows():
    nombre_completo = row['Nombre Funcionario'].split()
    nombre = nombre_completo[0]
    apellido = " ".join(nombre_completo[1:])
    # Verificar si la persona ya existe
    cursor.execute("SELECT 1 FROM persona WHERE rut = %s", (str(row['Rut']),))
    if cursor.fetchone() is None:
        insert_persona_query = sql.SQL("""
            INSERT INTO persona (name, rut, dv, lastname, position)
            VALUES (%s, %s, %s, %s, %s)
        """)
        cursor.execute(insert_persona_query, (
            nombre,
            str(row['Rut']),  # Convertir a cadena
            row['Dv'],
            apellido,
            row['Descripción Cargo']
        ))

# Insertar datos en la tabla departamento a partir del DataFrame
departamentos = df['Descripción Unidad'].unique()
for departamento in departamentos:
    # Verificar si el departamento ya existe
    cursor.execute("SELECT 1 FROM departamento WHERE name = %s", (departamento,))
    if cursor.fetchone() is None:
        insert_departamento_query = sql.SQL("""
            INSERT INTO departamento (name)
            VALUES (%s)
        """)
        cursor.execute(insert_departamento_query, (departamento,))

# Insertar datos en la tabla persona_departamento a partir del DataFrame
for _, row in df.iterrows():
    # Obtener el ID de la persona
    cursor.execute("SELECT id FROM persona WHERE rut = %s", (str(row['Rut']),))
    id_persona = cursor.fetchone()
    
    # Obtener el ID del departamento
    cursor.execute("SELECT id FROM departamento WHERE name = %s", (row['Descripción Unidad'],))
    id_departamento = cursor.fetchone()
    
    # Determinar si es director basado en la descripción de la función
    es_director = 'Jefe' in row['Descripción Función'] or 'Director' in row['Descripción Función']
    
    if id_persona and id_departamento:
        # Verificar si la relación ya existe para evitar duplicados
        cursor.execute("""
            SELECT 1 FROM persona_departamento WHERE id_persona = %s AND id_departamento = %s
        """, (id_persona[0], id_departamento[0]))
        if cursor.fetchone() is None:
            insert_persona_departamento_query = sql.SQL("""
                INSERT INTO persona_departamento (id_persona, id_departamento, es_director)
                VALUES (%s, %s, %s)
            """)
            cursor.execute(insert_persona_departamento_query, (id_persona[0], id_departamento[0], es_director))

# Insertar datos en la tabla staff a partir del DataFrame
staffs = df['Descripción Unidad 2'].unique()
for staff in staffs:
    # Verificar si el staff ya existe
    cursor.execute("SELECT 1 FROM staff WHERE name = %s", (staff,))
    if cursor.fetchone() is None:
        insert_staff_query = sql.SQL("""
            INSERT INTO staff (name)
            VALUES (%s)
        """)
        cursor.execute(insert_staff_query, (staff,))

# Insertar datos en la tabla staff_persona a partir del DataFrame
for _, row in df.iterrows():
    # Obtener el ID de la persona
    cursor.execute("SELECT id FROM persona WHERE rut = %s", (str(row['Rut']),))
    id_persona = cursor.fetchone()
    
    # Obtener el ID del staff
    cursor.execute("SELECT id FROM staff WHERE name = %s", (row['Descripción Unidad 2'],))
    id_staff = cursor.fetchone()
    
    if id_persona and id_staff:
        # Verificar si la relación ya existe para evitar duplicados
        cursor.execute("""
            SELECT 1 FROM staff_persona WHERE id_staff = %s AND id_persona = %s
        """, (id_staff[0], id_persona[0]))
        if cursor.fetchone() is None:
            insert_staff_persona_query = sql.SQL("""
                INSERT INTO staff_persona (id_staff, id_persona)
                VALUES (%s, %s)
            """)
            cursor.execute(insert_staff_persona_query, (id_staff[0], id_persona[0]))

# Confirmar los cambios
df.drop_duplicates(subset=["Rut"], keep="first")
connection.commit()

# Verificar datos en la tabla persona, departamento y staff
cursor.execute("SELECT * FROM persona")
personas = cursor.fetchall()
for persona in personas:
    print(persona)

cursor.execute("SELECT * FROM departamento")
departamentos = cursor.fetchall()
for departamento in departamentos:
    print(departamento)

cursor.execute("SELECT * FROM staff")
staffs = cursor.fetchall()
for staff in staffs:
    print(staff)

# Cerrar la conexión
cursor.close()
connection.close()



(1, 'Juan', None, None, 'Perez', 'Manager')
(2, 'Maria', None, None, 'Lopez', 'Director')
(3, 'Carlos', None, None, 'Garcia', 'Analyst')
(1990, 'CARO', '7459669', 'K', 'ANDIA MARTA ERCIRA DEL CARMEN', 'DIRECTOR DE SERVICIO')
(1991, 'AVACA', '7800155', '0', 'BALADRON MILITZA LUZ MARIA', 'ENFERMERA(O)')
(1992, 'ROJAS', '8000513', 'K', 'PINO OSCAR DANIEL', 'ENFERMERA(O)')
(1993, 'MEDINA', '8050617', '1', 'ESPAÑA JORGE ANTONIO', 'PROFESIONALES UNIVERSITARIOS')
(1994, 'FIGUEROA', '8140744', '4', 'JARAMILLO GRICELDA DE', 'OTROS TECNICOS')
(1995, 'COFRE', '8156924', 'K', 'ADASME CECILIA DEL CARMEN', 'ADMINISTRATIVO')
(1996, 'ROJAS', '8222726', '1', 'DIAZ DAVID DOMINGO', 'AUXILIAR')
(1997, 'PEÑA', '8289448', '9', 'FIGUEROA CHRISTIAN TOMAS', 'NUTRICIONISTA')
(1998, 'PEÑA', '8355634', 'K', 'OLIVOS JOSE LUIS', 'CHOFER')
(1999, 'ROJAS', '8497854', 'K', 'DIAZ MARTA ROSA', 'CONTADOR')
(2000, 'ARAYA', '8508850', '5', 'ARAYA IVAN ALFREDO', 'ASISTENTE SOCIAL')
(2001, 'ALBERTI', '8605290', '3', 'AYALA C

In [ ]:

def get_data_by_rut(rut):
    digito_verificador = rut % 10
    rut = rut // 10
    print(digito_verificador)
    return df[(df['Rut'] == rut) & (df['Dv'] == digito_verificador)]

In [26]:
get_data_by_rut(198084506)

6


,Rut,Dv,Correlativo,Nombre Funcionario,Código planta,Descripción Planta,Código Calidad Jurídica,Descripción Calidad Jurídica,Grado,Género,...,Establecimiento,Descripción Establecimiento,Fecha Nacimiento,Código Unidad,Descripción Unidad,Código Unidad 2,Descripción Unidad 2,Descripción Cargo,Descripción Función,Espec./Carrera
923,19808450,6,1,SOTO LETELIER JAVIER ALONSO,2,PROFESIONALES,22,CONTRATADOS,15,M,...,701,DSS. MAULE,1998-08-28,1370,DEPTO TECNOLOGIAS DE LA INFORMACION,1370,DEPTO TECNOLOGIAS DE LA INFORMACION,JEFE DEPTO. INFORMATICA,Apoyo Administrativo,s/d


In [10]:
df

,Rut,Dv,Correlativo,Nombre Funcionario,Código planta,Descripción Planta,Código Calidad Jurídica,Descripción Calidad Jurídica,Grado,Género,...,Establecimiento,Descripción Establecimiento,Fecha Nacimiento,Código Unidad,Descripción Unidad,Código Unidad 2,Descripción Unidad 2,Descripción Cargo,Descripción Función,Espec./Carrera
0,7459669,K,1,CARO ANDIA MARTA ERCIRA DEL CARMEN,1,DIRECTIVOS,21,TITULARES,2,F,...,701,DSS. MAULE,1955-04-04,1100,DIRECCION,1100,DIRECCION,DIRECTOR DE SERVICIO,Director DAP,s/d
1,7800155,0,1,AVACA BALADRON MILITZA LUZ MARIA,2,PROFESIONALES,21,TITULARES,12,F,...,701,DSS. MAULE,1956-01-11,1971,SAMUTALCA,1302,DEPARTAMENTO RED DE URGENCIA,ENFERMERA(O),ENFERMERA(O),ENFERMERA
2,8000513,K,1,ROJAS PINO OSCAR DANIEL,2,PROFESIONALES,21,TITULARES,7,M,...,701,DSS. MAULE,1956-07-14,1970,SAMUCURICO,1302,DEPARTAMENTO RED DE URGENCIA,ENFERMERA(O),ENFERMERA(O),ESPEC. EN ENFERMERIA MEDICA
3,8050617,1,1,MEDINA ESPAÑA JORGE ANTONIO,2,PROFESIONALES,21,TITULARES,8,M,...,701,DSS. MAULE,1965-04-14,1371,DESARROLLO DE SISTEMAS DE INFORMACION,1370,DEPTO TECNOLOGIAS DE LA INFORMACION,PROFESIONALES UNIVERSITARIOS,PROFESIONAL Y TECNICO UNIVERSITARIO,INGENIERO INFORMATICO
4,8140744,4,1,FIGUEROA JARAMILLO GRICELDA DE,3,TECNICOS,22,CONTRATADOS,15,F,...,701,DSS. MAULE,1961-08-15,1913,TESORERIA,1910,DEPARTAMENTO RECURSOS FINANCIEROS,OTROS TECNICOS,Apoyo Administrativo,s/d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,26756891,K,4,CARRERO RIVERO LUIS GUILLERMO,1,MEDICOS,12,CONTRATADOS,0,M,...,701,DSS. MAULE,1989-05-28,1318,SAMU CENTRO REGULADOR,1302,DEPARTAMENTO RED DE URGENCIA,M.C.MED.INTEGRAL,Atencion Clinica Medica,s/d
989,26792915,7,2,GUZÑAY SOLORZANO TONY FABIAN,1,MEDICOS,12,CONTRATADOS,,M,...,701,DSS. MAULE,1991-04-08,1401,BECA PRIMARIA,99-701,DSS. MAULE - BECARIOS,BECADOS PRIMARIOS,Medico Becario Primario,s/d
990,26796092,5,1,PIÑA MANZANILLA ZURIEL SILVETTE,1,MEDICOS,12,CONTRATADOS,0,F,...,701,DSS. MAULE,1992-06-13,1401,BECA PRIMARIA,99-701,DSS. MAULE - BECARIOS,BECADOS PRIMARIOS,Medico Becario Primario,s/d
991,26825161,8,1,YEPEZ SUAREZ KARLA ANDREINA,1,MEDICOS,12,CONTRATADOS,0,F,...,701,DSS. MAULE,1990-04-18,1401,BECA PRIMARIA,99-701,DSS. MAULE - BECARIOS,BECADOS PRIMARIOS,Medico Becario Primario,s/d
